# ADS Project 4: Machine Learning Fairness
## Spring 2023

In [1]:
# ! pip install dalex
import pandas as pd
import numpy as np
import dalex as dx
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from A7_algorithm_function import *

+ The cleaned COMPAS dataset is provided in `../output/compas-scores-two-years_cleaned.csv`. 

### Section 1: Load Data

In [2]:
df =  pd.read_csv('../output/compas-scores-two-years_cleaned.csv')
df.head(3)

,sex,age_cat,race,priors_count,c_charge_degree,two_year_recid,length_of_stay
0,Male,25 - 45,African-American,-0.733607,F,1,-0.167773
1,Male,< 25,African-American,0.055928,F,1,-0.340654
2,Male,25 - 45,Caucasian,2.029767,F,1,-0.244609


In [3]:
# change categorical data into numerical
le = LabelEncoder()
df['sex'] = le.fit_transform(df['sex'])
df['age_cat'] = le.fit_transform(df['age_cat'])
df['race'] = le.fit_transform(df['race'])
df['c_charge_degree'] = le.fit_transform(df['c_charge_degree'])

Encode categorical variables with numerical variables:
+ `sex`: 1 for male and 0 for female
+ `age_cat`: 2 for > 45, 0 for 25 - 45 and 1 for < 25
+ `race`: 1 for caucasian and 0 for african-american
+ `c_charge_degree`: 0 for F and 1 for M

In [4]:
# 
df['sex'].value_counts(), df['age_cat'].value_counts(), df['race'].value_counts(), df['c_charge_degree'].value_counts()

(1    4751
 0    1164
 Name: sex, dtype: int64,
 0    3378
 1    1281
 2    1256
 Name: age_cat, dtype: int64,
 0    3537
 1    2378
 Name: race, dtype: int64,
 0    3904
 1    2011
 Name: c_charge_degree, dtype: int64)

In [34]:
features = df[['sex', 'age_cat', 'c_charge_degree', 'length_of_stay',"priors_count"]]
sensitive = df['race']
target = df['two_year_recid']
X_train, X_test, y_train, y_test, race_train, race_test = \
    train_test_split(features, target, sensitive, test_size=0.3, random_state=6, shuffle = True)

+ **Protected**: Caucasians (i.e., `race == 1`)
+ **Not protected**: African-Americans (i.e., `race == 0`)

In [35]:
df.head()

,sex,age_cat,race,priors_count,c_charge_degree,two_year_recid,length_of_stay
0,1,0,0,-0.733607,0,1,-0.167773
1,1,1,0,0.055928,0,1,-0.340654
2,1,0,1,2.029767,0,1,-0.244609
3,0,0,1,-0.733607,1,0,-0.321445
4,1,1,1,-0.536224,0,1,-0.359864


The p-rule  function is commonly used in evaluate fairness in machine learning model, by checking whether the model's positive predictions are distributed similarly across different sensitive groups. The higher the p-rule, the better the fairness.

In [36]:
# Function to compute p-rule
def p_rule(sensitive_var, y_pred):
    protected = np.where(sensitive_var == 1)[0]
    not_protected = np.where(sensitive_var == 0)[0]
    protected_pred = np.where(y_pred[protected] == 1)
    not_protected_pred = np.where(y_pred[not_protected] == 1)
    protected_percent = protected_pred[0].shape[0]/protected.shape[0]
    not_protected_percent = not_protected_pred[0].shape[0]/not_protected.shape[0]
    ratio = min(protected_percent/not_protected_percent, not_protected_percent/protected_percent)
    
    return ratio, protected_percent, not_protected_percent

### Section 2: Logistic Regression
#### 2.1 Baseline Model
First, we train a baseline model without constaint, and then evaluate it's accuaracy and fairness.

In [37]:
clf = LogisticRegression(random_state= 6).fit(X_train, y_train)
coeff = clf.coef_
intercept = clf.intercept_
optimal_loss = log_loss(y_train, clf.predict_proba(X_train))
print(f'The optimal loss of logistic model is: {optimal_loss}')

The optimal loss of logistic model is: 0.6407097826469522


+ Accuaracy & Fairness check:

In [38]:
results_lr = {"Classifier": ["LR", "LR"], 
              "Set": ["Train", "Test"],
              "Accuracy (%)": [clf.score(X_train, y_train)*100, clf.score(X_test, y_test)*100],
              "P-rule (%)": [p_rule(race_train, clf.predict(X_train))[0]*100, p_rule(race_test, clf.predict(X_test))[0]*100],
              "Protected (%)": [p_rule(race_train, clf.predict(X_train))[1]*100, p_rule(race_test, clf.predict(X_test))[1]*100],
              "Not protected (%)": [p_rule(race_train, clf.predict(X_train))[2]*100, p_rule(race_test, clf.predict(X_test))[2]*100]}
pd.DataFrame(results_lr)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%)
0,LR,Train,63.743961,55.108121,21.962896,39.854192
1,LR,Test,62.028169,59.382529,24.186704,40.730337


#### 2.2 Optimizing classifier accuracy subject to fairness constraints

In [10]:
# Fairness Constraints: Mechanisms for Fair Classification

### Section 3: Support Vector Machine (SVM) 
#### 3.1 Baseline Model

In [39]:
from sklearn import svm
from sklearn.svm import SVC

# Train model
svm_model = SVC(kernel = 'linear', probability = True)
clf= svm_model.fit(X_train, y_train)
optimal_loss = log_loss(y_train, clf.predict_proba(X_train))
print(f'The optimal loss of SVM model is: {optimal_loss}')

The optimal loss of SVM model is: 0.6445239963245335


In [40]:
# Display results
results_svm = {"Classifier": ["SVM", "SVM"],
               "Set": ["Train", "Test"],
               "Accuracy (%)": [clf.score(X_train, y_train)*100, clf.score(X_test, y_test)*100],
               "P-rule (%)": [p_rule(race_train, clf.predict(X_train))[0]*100, p_rule(race_test, clf.predict(X_test))[0]*100],
               "Protected (%)": [p_rule(race_train, clf.predict(X_train))[1]*100, p_rule(race_test, clf.predict(X_test))[1]*100],
               "Not protected (%)": [p_rule(race_train, clf.predict(X_train))[2]*100, p_rule(race_test, clf.predict(X_test))[2]*100]}
pd.DataFrame(results_svm)

,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%)
0,SVM,Train,62.874396,51.242571,15.918612,31.065209
1,SVM,Test,61.352113,53.767411,17.821782,33.146067


#### 3.2 Optimizing SVM classifier accuracy subject to fairness constraints

In [13]:
# Fairness Constraints: Mechanisms for Fair Classification

### Section 4: Information Theoretic Measures for Fairness-aware Feature Selection(FFS) 

An another method to deal with machine learning fairness is called **Information Theoretic Measures for Fairness-aware Feature selection (FFS)**. 

In short, from the joint statistics of the data, the framework proposes that two information theoretic measures can be used to quantify the **accuracy** and **discrmination** aspect for each subset of the feature space. We then compute the **Shapley coefficients** for each feature to capture its effect on the sensitive/protected group.

Below is how we choose features by using **FFS** Algorithm.

In [17]:
y=df["two_year_recid"]
y=y.to_numpy()
y=np.reshape(y,(-1,1))
a=df["race"]
a=a.to_numpy()
a=np.reshape(a,(-1,1))
arr=["sex","age_cat","priors_count","c_charge_degree","length_of_stay"]
for i in arr:
    print("Feature:",end="")
    print(i)
    print("Marginal Accuracy Coefficient:",end="")
    print(shapley_accuracy(i,y,a))
    print("Marginal Discrimination Coefficient:",end="")
    print(shapley_discrimination(i,y,a))

Feature:sex
Marginal Accuracy Coefficient:0.003568314907946012
Marginal Discrimination Coefficient:8.16082890138538e-06
Feature:age_cat
Marginal Accuracy Coefficient:0.011273246895526084
Marginal Discrimination Coefficient:4.4881509198872635e-05
Feature:priors_count
Marginal Accuracy Coefficient:0.02427159853758174
Marginal Discrimination Coefficient:4.464995590653898e-05
Feature:c_charge_degree
Marginal Accuracy Coefficient:0.001958893676796756
Marginal Discrimination Coefficient:6.936830123648318e-06
Feature:length_of_stay
Marginal Accuracy Coefficient:0.005168319175750994
Marginal Discrimination Coefficient:1.0984742883746511e-05


In [27]:
# display result
shapley_results = pd.DataFrame()
shapley_results["Feature"] = arr
shapley_results["Accuracy Coefficient"] = [0.003568314907946012, 0.011273246895526084,0.02427159853758174,0.001958893676796756,0.005168319175750994]
shapley_results["Discrimination Coefficient"] = [8.16082890138538e-06,4.4881509198872635e-05,4.464995590653898e-05,6.936830123648318e-06,1.0984742883746511e-05]
shapley_results

,Feature,Accuracy Coefficient,Discrimination Coefficient
0,sex,0.003568,0.000008
1,age_cat,0.011273,0.000045
2,priors_count,0.024272,0.000045
3,c_charge_degree,0.001959,0.000007
4,length_of_stay,0.005168,0.000011


From the table above, we can see that **Discrimination Coefficient** of `priors_count` is highest, but **Accuracy Coefficient** is also high, so we can't ignore this feature. `length_of_stay` has a little high **Discrimination Coefficient**, and its **Accuracy Coefficient** is low, so we can ignore this feature. So finally, we can choose these four features:

- `priors_count`
- `age_cat`
- `sex`
- `c_charge_degree`

#### 4.1 Logistic Regression using FFS

In [46]:
features = df[['sex', 'age_cat', 'c_charge_degree', 'priors_count']]
sensitive = df['race']
target = df['two_year_recid']
X_train, X_test, y_train, y_test, race_train, race_test = \
    train_test_split(features, target, sensitive, test_size=0.3, random_state=6, shuffle = True)

In [47]:
# Train model and print results
clf = LogisticRegression(random_state = 0).fit(X_train, y_train)
coeff = clf.coef_
intercept = clf.intercept_
optimal_loss = log_loss(y_train, clf.predict_proba(X_train))
print(f'The optimal loss of logistic model is: {optimal_loss}')

# Display results
results_ffs_lr = {"Classifier": ["FFS-LR", "FFS-LR"],
                  "Set": ["Train", "Test"],
                  "Accuracy (%)": [clf.score(X_train, y_train)*100, clf.score(X_test, y_test)*100],
                  "P-rule (%)": [p_rule(race_train, clf.predict(X_train))[0]*100, p_rule(race_test, clf.predict(X_test))[0]*100],
                  "Protected (%)": [p_rule(race_train, clf.predict(X_train))[1]*100, p_rule(race_test, clf.predict(X_test))[1]*100],
                  "Not protected (%)": [p_rule(race_train, clf.predict(X_train))[2]*100, p_rule(race_test, clf.predict(X_test))[2]*100]}
pd.DataFrame(results_ffs_lr)

The optimal loss of logistic model is: 0.6436511330361576


,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%)
0,FFS-LR,Train,63.405797,58.808292,23.937762,40.704739
1,FFS-LR,Test,61.802817,65.541470,26.449788,40.355805


#### 4.2 SVM using FFS

In [45]:
# Train model
svm_model = SVC(kernel = 'linear', probability = True)
clf= svm_model.fit(X_train, y_train)
optimal_loss = log_loss(y_train, clf.predict_proba(X_train))
print(f'The optimal loss of SVM model is: {optimal_loss}')

# Display results
results_svm = {"Classifier": ["SVM", "SVM"],
               "Set": ["Train", "Test"],
               "Accuracy (%)": [clf.score(X_train, y_train)*100, clf.score(X_test, y_test)*100],
               "P-rule (%)": [p_rule(race_train, clf.predict(X_train))[0]*100, p_rule(race_test, clf.predict(X_test))[0]*100],
               "Protected (%)": [p_rule(race_train, clf.predict(X_train))[1]*100, p_rule(race_test, clf.predict(X_test))[1]*100],
               "Not protected (%)": [p_rule(race_train, clf.predict(X_train))[2]*100, p_rule(race_test, clf.predict(X_test))[2]*100]}
pd.DataFrame(results_svm)

The optimal loss of SVM model is: 0.648341294279551


,Classifier,Set,Accuracy (%),P-rule (%),Protected (%),Not protected (%)
0,SVM,Train,62.512077,51.714542,15.918612,30.781693
1,SVM,Test,61.183099,57.733015,18.811881,32.584270


### Section 5: Evaluation